Using BERTopic for Topic Modeling

In [1]:
!pip install BERTopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.1 MB/s eta 0:00:00
  Preparing metadata 

In [2]:
from bertopic import BERTopic


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/gdrive/MyDrive/tokyo_2020_tweets.csv')
df = df.dropna()
df = df.sample(20000, random_state=42)
df.head()

<ipython-input-4-36677e9a3517>:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/gdrive/MyDrive/tokyo_2020_tweets.csv')


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.text.apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,clean_text
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False,congrats 🇦🇺 🇧🇪 🇱🇺 achievements far olympics 👏👏👏
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False,reality vs new york times
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False,live score wd group greysia polii apriyani ra...
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False,tennis tokyo olympics stefanos becomes first g...
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False,year old gold medalist ethnic kurd western ir...


In [ ]:
tweets = df.clean_text.to_list()
timestamp = df.date.to_list()

In [5]:
tweets = df.text.to_list()
timestamp = df.date.to_list()

In [6]:
tweets[0:5]

['Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievements so far at the #Tokyo2020 Olympics! 👏👏👏 https://t.co/8GnCXWdY5g',
 'Reality            Vs      New York Times\n#Tokyo2020 https://t.co/fvtjqRy6q1',
 '[Live Score] #Tokyo2020\xa0\xa0 \n\nWD Group A : \n\nGreysia Polii/Apriyani Rahayu [INA] vs Chloe Birch/Lauren Smith [GBR]\n\n2… https://t.co/QbLuL0W6G5',
 'Tennis at Tokyo Olympics: Stefanos #Tsitsipas becomes the first Greek man to win an Olympic singles match in 97 yea… https://t.co/W12eEPcp70',
 "The 41-year-old gold medalist is an ethnic Kurd from western Iran's Illam province. #Tokyo2020 #Olympics… https://t.co/jGnD12HW9C"]

In [7]:
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [8]:
topics, probs = topic_model.fit_transform(tweets)

In [9]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7097,-1_https_co_tokyo2020_the,"[https, co, tokyo2020, the, to, olympics, for,...",[It's almost time for the Quarter-Finals! \n\n...
1,0,330,0_mirabaichanu_silver_mirabai_chanu_winning,"[mirabaichanu, silver, mirabai_chanu, winning,...",[Congratulations to Saikhom Mirabai Chanu For ...
2,1,293,1_olympics_olympicgames_sport_games,"[olympics, olympicgames, sport, games, olympic...","[Are you watching the #Tokyo2020 Olympics?, Wh..."
3,2,267,2_coverage_bbc_live_tv,"[coverage, bbc, live, tv, gt, nbc, watch, bbcs...","[#Tokyo2020 \nGood Coverage At The Moment., Ti..."
4,3,263,3_weightlifting_silver_chanu_mirabai,"[weightlifting, silver, chanu, mirabai, weight...",[First medal for India at #Tokyo2020 #Olympi...
...,...,...,...,...,...
270,269,10,269_hockey_yktxakbjyz_skirts_sawomenshockey,"[hockey, yktxakbjyz, skirts, sawomenshockey, a...","[What our Women’s team can do, our Men’s team ..."
271,270,10,270_crying_tears_emotional_happy,"[crying, tears, emotional, happy, 0nr5qxs83m, ...",[Watching the @MichaelPhelps Olympic highlight...
272,271,10,271_daley_tom_matty_lee,"[daley, tom, matty, lee, never, yessssssss, ps...",[Tom Daley and Matty Lee making me late for wo...
273,272,10,272_owen_wright_surfing_bronze,"[owen, wright, surfing, bronze, 5visu33p7r, w8...",[Bronze medal!🥉Congrats Owen Wright for Austra...


In [10]:
topic_model.get_topic(0)

[('mirabaichanu', 0.03523250506063375),
 ('silver', 0.02904783144720042),
 ('mirabai_chanu', 0.02686225106178668),
 ('winning', 0.023605270664785353),
 ('medal', 0.022046774905446948),
 ('india', 0.022004619117713186),
 ('congratulations', 0.020718567481166447),
 ('mirabai', 0.019687504116753067),
 ('chanu', 0.018671592401198672),
 ('first', 0.013504610357324755)]

In [11]:
topic_model.get_representative_docs(0)

['Congratulations to Saikhom Mirabai Chanu For opening India’s medals tally at #Tokyo2020   with a Silver medal. 🇮🇳 #Cheer4India #MirabaiChanu',
 'Congratulations #MirabaiChanu on winning Silver medal for the country! 🇮🇳🥈\n\n#Tokyo2020',
 '￼Many Congratulations to \xa0#TeamIndia @mirabai_chanu \nfor winning silver medal  #Olympics\xa0￼ \n\n#Tokyo2020']

In [12]:
df=pd.DataFrame({"topics":topics,"documents":tweets})
df

,topics,documents
0,-1,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement..."
1,15,Reality Vs New York Times\n#To...
2,184,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...
3,-1,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...
4,-1,The 41-year-old gold medalist is an ethnic Kur...
...,...,...
19995,-1,#Hockey\nNot expected results. Men in Blue los...
19996,2,@BBCOne The final is set to take place today a...
19997,0,Congratulations to the superwoman #MirabaiChan...
19998,-1,@gretchenho Valdez now at 45th out of 47. Less...


In [13]:
topic_model.visualize_topics()

In [15]:
topic_model.visualize_barchart(n_words = 15)

In [18]:
topic_model.visualize_hierarchy()